<a href="https://colab.research.google.com/github/mr-ankit-tech/Coding_with_llm/blob/main/Super_Agent_Web_GUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# === REQUIRED INSTALLS ===
!pip install langchain langchain_openai langchain_community
!pip install transformers accelerate bitsandbytes
!pip install huggingface_hub
!pip install langchain-huggingface
!pip install matplotlib
!pip install gradio matplotlib


# === PYTHON IMPORTS ===
import os, sqlite3
import pandas as pd
import io
import textwrap
from IPython.display import display
from datetime import datetime
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.sql_database import SQLDatabase
from langchain.prompts import PromptTemplate
from google.colab import userdata, data_table
import matplotlib.pyplot as plt
from functools import wraps
import gradio as gr

In [ ]:

# Define the desired width for your lines (e.g., 80 characters)
wrap_width = 80

# === LOAD DATA INTO SQLITE ===
db_path = os.path.abspath("ankit_tutorial.db")
sqlite_connection = sqlite3.connect(db_path)

for csv_file, table_name in [("/content/Heartbeat.csv", "Heartbeat"), ("/content/Audit.csv", "Audit")]:
    df = pd.read_csv(csv_file)
    df.to_sql(table_name, sqlite_connection, if_exists="replace", index=False)

sqlite_connection.close()

# === INIT OpenAI + LangChain DB ===
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name="gpt-3.5-turbo-1106")
my_db = SQLDatabase.from_uri(f"sqlite:///{db_path}")


# === MEMORY ===
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# === PROMPT FOR COMPLEX SQL GENERATION ===
sql_prompt_template = PromptTemplate(
    input_variables=["chat_history","user_inquiry", "background_info"],
    template="""
You are a senior data analyst writing optimized sql queries.

Rules:
- Never use SELECT * — only necessary columns.
- Use table aliases when helpful.
- You can use:
  - window functions (RANK, ROW_NUMBER)
  - CTEs (WITH ...)
  - CASE statements
  - GROUP BY aggregations
- Actual rate = Actual_Revenue / Actual_FFE
- Booked rate = Booked_Revenue / Booked_FFE
- Volume = FFE

Schema:
{background_info}


# Chat History:
 {chat_history}


Question: {user_inquiry}
Return only the SQL query.
"""
)

# === FETCH SCHEMA ===
def get_table_schema(table_name: str) -> str:
    if table_name not in my_db.get_table_names():
        raise ValueError(f"Table '{table_name}' does not exist.")
    return my_db.get_table_info([table_name])

# === INSIGHT SUMMARY ===
def generate_insight(user_inquiry, df_result):
    prompt = f"""
        User asked: {user_inquiry}
        Here's the data (top 10 rows):
        {df_result.head(10).to_markdown(index=False)}
        Please summarize any key trends, patterns, or insights in simple language:
        """
    return llm.invoke(prompt).content.strip()

# === FOLLOW-UP SUGGESTIONS ===
def suggest_follow_ups(user_inquiry, df_result):
    prompt = f"""
        Given the result for: {user_inquiry}
        And top rows of the data:
        {df_result.head(5).to_markdown(index=False)}
        Suggest 2-3 intelligent follow-up questions for deeper exploration:
      """
    return llm.invoke(prompt).content.strip()


# === MAIN SUPER AGENT FUNCTION ===
def Super_Agent_Dashboard(table_name: str, user_inquiry: str) -> Tuple[str, pd.DataFrame, Optional[str], str, str, str, str]:
    try:
        # Step 1: Fetch Schema
        table_schema = get_table_schema(table_name)
        background_info = f"Table `{table_name}` has the following schema:\n{table_schema}"

        # Step 2: Generate SQL
        # Corrected indentation for the following lines
        formatted_prompt = sql_prompt_template.format(
            background_info=background_info,
            user_inquiry=user_inquiry,
            chat_history=memory.load_memory_variables({})["chat_history"]
        )
        sql_query = llm.invoke(formatted_prompt).content.strip()
        sql_query = str(sql_query)

        # Optional: sanity check
        if not (sql_query.lower().startswith("select") or sql_query.lower().startswith("with")):
          raise ValueError(f"Generated content is not SQL:\n{sql_query}")

        # Step 3: Execute Query
        conn = sqlite3.connect(db_path)
        df_result = pd.read_sql_query(sql_query, conn)
        conn.close()

        # Visualize
        plot_img = None
        if df_result.shape[1] == 2 and pd.api.types.is_numeric_dtype(df_result.iloc[:, 1]):
            plt.figure()
            df_result.plot(kind='bar', x=df_result.columns[0], y=df_result.columns[1], legend=False)
            plt.xticks(rotation=45)
            plt.tight_layout()
            plot_path = f"/tmp/barplot_{datetime.now().strftime('%H%M%S')}.png"
            plt.savefig(plot_path)
            plot_img = plot_path
            plt.close()

        # Step 4: Save to CSV
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        fname = f"Extract_{table_name}_{timestamp}.csv"
        df_result.to_csv(fname, index=False)
        # print(f"📁 Saved to: {fname}")

        # Step 5: Generate Insight and follow-ups
        insight = generate_insight(user_inquiry, df_result)
        followups = suggest_follow_ups(user_inquiry, df_result)

        # Update memory with the interaction (Question and Insight)
        memory.chat_memory.add_user_message(user_inquiry)
        memory.chat_memory.add_ai_message(insight)

        #print("🔍 Insight:\n", textwrap.fill(insight, width=wrap_width))
        #print("\n🤖 Follow-Up Suggestions:\n", followups)

        status = f"✅ Query executed successfully. Saved as: {fname}"
        return (
            sql_query,
            df_result,
            plot_img,
            insight,
            followups,
            status,
            fname
        )

    except Exception as e:
      import traceback
      error_msg = traceback.format_exc()
      return "Error", pd.DataFrame(), None, "", "", f"❌ Error:\n{str(e)}\n\n📄 Trace:\n{error_msg}"

In [ ]:
# Dropdown from tables
table_names = my_db.get_table_names()

with gr.Blocks(title="Dynamic Agent Dashboard") as dashboard:
    gr.Markdown("## 📊 Dynamic Agent Dashboard")
    gr.Markdown("Query your data with natural language.")

    with gr.Row():
        table_dropdown = gr.Dropdown(choices=table_names, label="Select Table", value=table_names[0])
        query_input = gr.Textbox(lines=2, label="Your Question")

    submit_btn = gr.Button("Run Query")
    sql_out = gr.Textbox(label=" 🧠 Generated SQL Query")
    df_out = gr.Dataframe(label=" 📄 Data Preview")
    chart_out = gr.Image(label=" 📊 Visualization (if applicable)",type="pil")
    insight_out = gr.Textbox(label=" 🔍 Insight Summary",lines=3)
    followup_out = gr.Textbox(label=" 🤖 Follow-Up Questions",lines=2)
    status_output = gr.Textbox(label="Log & Status", lines=2)


    # Hook up backend
    submit_btn.click(
        fn=Super_Agent_Dashboard,
        inputs=[table_dropdown, query_input],
        outputs=[sql_out, df_out, chart_out, insight_out, followup_out, status_output]
        )
dashboard.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3d6f5b6280a62f8f1b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:

# Dropdown from tables
table_names = my_db.get_table_names()

with gr.Blocks(title="Dynamic Agent Dashboard") as demo:
    gr.Markdown("## 📊 Dynamic Agent Dashboard")
    gr.Markdown("Query your data with natural language.")

    with gr.Row():
        table_dropdown = gr.Dropdown(choices=table_names, label="Select Table", value=table_names[0])
        query_input = gr.Textbox(lines=2, label="Your Question")

    submit_btn = gr.Button("Run Query")

    gr.Markdown("### 🧠 Generated SQL Query")
    sql_out = gr.Textbox(label="SQL", lines=4)

    gr.Markdown("### 📄 Data Preview")
    df_out = gr.Dataframe()

    gr.Markdown("### 📊 Visualization (if applicable)")
    chart_out = gr.Image(type="pil")

    gr.Markdown("### 🔍 Insight Summary")
    insight_out = gr.Textbox(lines=3)

    gr.Markdown("### 🤖 Follow-Up Questions")
    followup_out = gr.Textbox(lines=2)

    gr.Markdown("### 🛠️ Status")
    status_output = gr.Textbox(label="Log & Status", lines=2)


    # Hook up backend
    submit_btn.click(
        fn=Super_Agent_Dashboard,
        inputs=[table_dropdown, query_input],
        outputs=[sql_out, df_out, chart_out, insight_out, followup_out, status_output]
        )


demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4d74444510ae98359f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import os
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from typing import Optional, Tuple
import gradio as gr
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.utilities import SQLDatabase
import textwrap
from google.colab import userdata

# === SETTINGS ===
wrap_width = 80
db_path = os.path.abspath("ankit_tutorial.db")

# === LOAD CSV DATA INTO SQLITE ===
sqlite_connection = sqlite3.connect(db_path)
for csv_file, table_name in [("/content/Heartbeat.csv", "Heartbeat"), ("/content/Audit.csv", "Audit")]:
    df = pd.read_csv(csv_file)
    df.to_sql(table_name, sqlite_connection, if_exists="replace", index=False)
sqlite_connection.close()

# === INIT LLM + LangChain SQL DB ===
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')  # or use secure input
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name="gpt-3.5-turbo-1106")
my_db = SQLDatabase.from_uri(f"sqlite:///{db_path}")
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# === PROMPT TEMPLATE FOR SQL ===
sql_prompt_template = PromptTemplate(
    input_variables=["chat_history", "user_inquiry", "background_info"],
    template="""
You are a senior data analyst writing optimized sql queries.

Rules:
- Never use SELECT * — only necessary columns.
- Use table aliases when helpful.
- You can use:
  - window functions (RANK, ROW_NUMBER)
  - CTEs (WITH ...)
  - CASE statements
  - GROUP BY aggregations
- Actual rate = Actual_Revenue / Actual_FFE
- Booked rate = Booked_Revenue / Booked_FFE
- Volume = FFE

Schema:
{background_info}

# Chat History:
{chat_history}

Question: {user_inquiry}
Return only the SQL query.
"""
)

# === GET TABLE SCHEMA ===
def get_table_schema(table_name: str) -> str:
    if table_name not in my_db.get_table_names():
        raise ValueError(f"Table '{table_name}' does not exist.")
    return my_db.get_table_info([table_name])

# === GENERATE INSIGHT FROM RESULT ===
def generate_insight(user_inquiry, df_result):
    prompt = f"""
User asked: {user_inquiry}
Here's the data (top 10 rows):
{df_result.head(10).to_markdown(index=False)}
Please summarize any key trends, patterns, or insights in simple language:
"""
    return llm.invoke(prompt).content.strip()

# === SUGGEST FOLLOW-UPS ===
def suggest_follow_ups(user_inquiry, df_result):
    prompt = f"""
Given the result for: {user_inquiry}
And top rows of the data:
{df_result.head(5).to_markdown(index=False)}
Suggest 2-3 intelligent follow-up questions for deeper exploration:
"""
    return llm.invoke(prompt).content.strip()

# === SUPER AGENT DASHBOARD ===
def Super_Agent_Dashboard(table_name: str, user_inquiry: str) -> Tuple[str, pd.DataFrame, Optional[str], str, str, str, str]:
    try:
        # Step 1: Get schema
        table_schema = get_table_schema(table_name)
        background_info = f"Table `{table_name}` has the following schema:\n{table_schema}"

        # Step 2: Generate SQL
        formatted_prompt = sql_prompt_template.format(
            background_info=background_info,
            user_inquiry=user_inquiry,
            chat_history=memory.load_memory_variables({})["chat_history"]
        )
        sql_query = llm.invoke(formatted_prompt).content.strip()

        # if not (sql_query.lower().startswith("select") or sql_query.lower().startswith("WITH")):
        #    raise ValueError(f"Generated content is not SQL:\n{sql_query}")

        if not (sql_query.lower().startswith("select") or sql_query.lstrip().lower().startswith("with")):
          raise ValueError(f"Generated content is not SQL:\n{sql_query}")

        # Step 3: Execute SQL
        conn = sqlite3.connect(db_path)
        df_result = pd.read_sql_query(sql_query, conn)
        conn.close()

        # Step 4: Generate Chart (optional)
        plot_img = None
        if df_result.shape[1] == 2 and pd.api.types.is_numeric_dtype(df_result.iloc[:, 1]):
            plt.figure()
            df_result.plot(kind='bar', x=df_result.columns[0], y=df_result.columns[1], legend=False)
            plt.xticks(rotation=45)
            plt.tight_layout()
            plot_path = f"/tmp/barplot_{datetime.now().strftime('%H%M%S')}.png"
            plt.savefig(plot_path)
            plot_img = plot_path
            plt.close()

        # Step 5: Save to CSV
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        fname = f"Extract_{table_name}_{timestamp}.csv"
        df_result.to_csv(fname, index=False)

        # Step 6: Generate insight & followups
        insight = generate_insight(user_inquiry, df_result)
        followups = suggest_follow_ups(user_inquiry, df_result)

        # Step 7: Update memory
        memory.chat_memory.add_user_message(user_inquiry)
        memory.chat_memory.add_ai_message(insight)

        status = f"✅ Query executed successfully. Saved as: {fname}"
        return sql_query, df_result, plot_img, insight, followups, status, fname

    except Exception as e:
        import traceback
        error_msg = traceback.format_exc()
        return "Error", pd.DataFrame(), None, "", "", f"❌ Error:\n{str(e)}\n\n📄 Trace:\n{error_msg}", ""

# === CHATBOT HANDLER ===
def handle_chat(user_message: str) -> Tuple[list, str]:
    try:
        memory.chat_memory.add_user_message(user_message)

        # Optionally: include history in prompt
        context_prompt = f"""
You are a data-savvy assistant helping the user explore their data.
Previous context:
{memory.load_memory_variables({})["chat_history"]}

User: {user_message}
"""
        ai_response = llm.invoke(context_prompt).content.strip()
        memory.chat_memory.add_ai_message(ai_response)

        chat_history = [(m.content, n.content) for m, n in zip(
            memory.chat_memory.messages[::2],
            memory.chat_memory.messages[1::2]
        )]
        return chat_history, ""
    except Exception as e:
        return [], f"❌ Error: {str(e)}"

# === GRADIO UI ===
table_names = my_db.get_table_names()

with gr.Blocks(title="Dynamic Agent Dashboard") as dashboard:
    gr.Markdown("## 📊 Dynamic Agent Dashboard")
    gr.Markdown("Query your data with natural language.")

    with gr.Row():
        table_dropdown = gr.Dropdown(choices=table_names, label="Select Table", value=table_names[0])
        query_input = gr.Textbox(lines=2, label="Your Question")
    submit_btn = gr.Button("Run Query")

    sql_out = gr.Textbox(label="🧠 Generated SQL Query")
    df_out = gr.Dataframe(label="📄 Data Preview")
    chart_out = gr.Image(label="📊 Visualization", type="pil")
    insight_out = gr.Textbox(label="🔍 Insight Summary", lines=3)
    followup_out = gr.Textbox(label="🤖 Follow-Up Suggestions", lines=2)
    status_output = gr.Textbox(label="Status Log", lines=2)

    # Chatbot UI
    gr.Markdown("### 💬 Continue the conversation")
    chatbot = gr.Chatbot(label="🧠 Data Chat Assistant")
    chat_input = gr.Textbox(label="Your message", lines=1, placeholder="Ask a follow-up...")
    chat_submit = gr.Button("Send")

    # Button hooks
    submit_btn.click(
        fn=Super_Agent_Dashboard,
        inputs=[table_dropdown, query_input],
        outputs=[sql_out, df_out, chart_out, insight_out, followup_out, status_output]
    )

    chat_submit.click(fn=handle_chat, inputs=[chat_input], outputs=[chatbot, status_output])

dashboard.launch(share=True)

/tmp/ipython-input-13-1460712987.py:191: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="🧠 Data Chat Assistant")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4a819a9896c581765a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
